In [1]:
import numpy as np
import pandas as pd
from __future__ import division

In [2]:
train_data = pd.read_csv("data/train_data.csv",sep='\t',header=None)
songs = pd.read_csv("data/songs.csv",sep='\t',header=None)
users = pd.read_csv("data/users.csv",sep='\t',header=None)
test_data = pd.read_csv("data/test_data.csv",sep='\t',header=None)

In [3]:
train_data = train_data.rename(columns={train_data.columns[0]:"user",train_data.columns[1]:"song",train_data.columns[2]:"number"})
test_data = test_data.rename(columns={test_data.columns[0]:"user",test_data.columns[1]:"song",test_data.columns[2]:"number"})
songs = songs.rename(columns={songs.columns[0]:"song"})
users = users.rename(columns={users.columns[0]:"user"})


In [4]:
users["user_id"] = ["user_" +str(i) for i in range(5000)]
songs["song_id"] = ["song_" +str(i) for i in range(49564)]

In [5]:
def transformation(data, users, songs):
    tmp =pd.merge(data,users,how='left',left_on=data.user, right_on=users.user)
    tmp1 = pd.merge(tmp,songs,how='left',left_on=tmp.song, right_on=songs.song)
    return tmp1[tmp1.columns[[4,6,2]]]

In [6]:
train_data = transformation(train_data,users,songs)
test_data = transformation(test_data, users, songs)

In [7]:
n,m = len(users),len(songs)

In [8]:
matrix = np.zeros((n,m))



In [9]:
train_data['rate'] = 1
train_data.rate[(train_data.number<=2)&(train_data.number>=1)] = 2

/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [97]:
train_data.rate[(train_data.number>2)&(train_data.number<=3)]=3
train_data.rate[(train_data.number>3)&(train_data.number<=5)]=4
train_data.rate[(train_data.number>5)&(train_data.number<=8)]=5
train_data.rate[(train_data.number>8)&(train_data.number<=12)]=6
train_data.rate[(train_data.number>12)&(train_data.number<=18)]=7
train_data.rate[(train_data.number>18)]=8
train_data

/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/yuxiang

,user_id,song_id,number,rate
0,user_3873,song_33722,1,2
1,user_3873,song_30052,1,2
2,user_3873,song_40941,5,4
3,user_3873,song_27675,1,2
4,user_3873,song_14539,3,3
5,user_3873,song_25141,1,2
6,user_3873,song_6887,19,8
7,user_3873,song_14598,2,2
8,user_3873,song_30457,1,2
9,user_3873,song_12558,6,5


In [153]:
def to_matrix_rate(data,matrix):
    matrix = np.zeros((n,m))


    for r in data.iterrows():
        tmp = r[1]
        row = int(tmp[0].split("_")[-1])
        col = int(tmp[1].split("_")[-1])
        rate = tmp[3]
        matrix[row][col] = rate
    return matrix

In [154]:
def to_matrix_number(data,matrix):
    matrix = np.zeros((n,m))


    for r in data.iterrows():
        tmp = r[1]
        
        row = int(tmp[0].split("_")[-1])
        col = int(tmp[1].split("_")[-1])
        number = tmp[2]
        matrix[row][col] = number
    return matrix

In [155]:
V_train = to_matrix_number(train_data,matrix)


In [157]:
max(V_train[0])

2.0

In [158]:
V_test = to_matrix_number(test_data, matrix)

In [159]:
train_data[train_data.user_id=="user_0"]

,user_id,song_id,number,rate
62601,user_0,song_11814,1,2
62602,user_0,song_36979,1,2
62603,user_0,song_32134,2,2
62604,user_0,song_9733,1,2
62605,user_0,song_2889,1,2
62606,user_0,song_23812,2,2
62607,user_0,song_33351,1,2
62608,user_0,song_11195,2,2
62609,user_0,song_22433,1,2
62610,user_0,song_14362,1,2


In [160]:
test_data[test_data.user_id=="user_0"]

,user_id,song_id,number
64934,user_0,song_6655,1
64935,user_0,song_48270,1
64936,user_0,song_42069,1
64937,user_0,song_45281,1
64938,user_0,song_23084,2
64939,user_0,song_19901,1
64940,user_0,song_21267,1
64941,user_0,song_37714,4
64942,user_0,song_7279,1
64943,user_0,song_13374,1


In [161]:
def indicate(matrix):
    index_row,index_col = np.nonzero(matrix)
    M = np.zeros(matrix.shape)
    for i in range(len(index_row)):
        M[index_row[i]][index_col[i]] = 1
    return M
def normalization(matrix, max_rating ):
    matrix +=1
    matrix[matrix>max_rating+1] = max_rating
    matrix[matrix<1] = 1
    matrix[matrix<0] = 1
    return matrix
def SVD_incremental(V_train,feature_number,iteration_number,regu = 0.001,max_rating =7, learning_rate = 0.005):
    n,m = V_train.shape
    U = np.ones((n, feature_number))*0.01
    M = np.ones((m, feature_number))*0.01
    I = indicate(V_train)
    for j in range(iteration_number):
        MUL = np.dot(U,M.T)
        MUL = normalization(MUL, max_rating)
        tmp = np.multiply((V_train - MUL),I)
        delta_U = []
        delta_M = []
        for i in range(n):
            delta_U.append(np.sum(tmp[i]*M.T,axis=1))
        delta_U= np.array(delta_U) -regu*U

        for k in range(m):
            delta_M.append(np.sum(tmp.T[k]*U.T,axis=1))
        delta_M= np.array(delta_M) -regu*M
        if j>200:
            learning_rate = 0.004
        if j>400:
            learning_rate = 0.003
        if j>600:
            learning_rate = 0.002
        if j>800:
            learning_rate = 0.001
        if j>1000:
            learning_rate = 0.0005
        if j>1500:
            learning_rate = 0.0001
        
        U = U + delta_U*learning_rate
        M = M + delta_M *learning_rate
        print error(V_train,U,M,I,regu),j
    return U,M
def error(V_train, U,M,I,regu):
    MUL = V_train-np.dot(U,M.T)
    error = np.sum(I*(MUL**2))
    error = error 
    error = error * 0.5
    return error 

In [162]:
def return_most_popular(prediction,n_popular):
    x = []
    n,m = prediction.shape
    for i in range(n):
        s = sorted(range(m), key=lambda k: prediction[i][k] ,reverse=True)
        x.append(s[:n_popular])
    x = np.array(x)
    return x

In [163]:
def most_popular(V_train, n_popular):
    I = indicate(V_train)
    stat = np.sum(I,axis=0)
    s = sorted(range(len(stat)), key=lambda k: stat[k] ,reverse=True)
    x = []
    for i in range(n):
        x.append(s[:n_popular])
    x = np.array(x)


    return x

In [164]:
recommendation = most_popular(V_train,n_popular=100)

In [165]:
def correct_rate(V_test,recommendation,n_rank):
    I = indicate(V_test)
    rate = 0
    for i in range(n_rank):
        
        rate +=rank_rate(I,recommendation,rank_number=n_rank)
        
    return rate/n_rank

def rank_rate(I,re,rank_number):
    k = 0
    for j in range(n):
        for i in re[j][:rank_number]:
            k +=I[j][i]
    return k/n/rank_number   

In [167]:
correct_rate(V_test,recommendation,n_rank=20)

0.027029999999999998

In [179]:
U = np.genfromtxt("U.txt",delimiter=',')
M = np.genfromtxt("M.txt",delimiter=',')

In [180]:
predict = np.dot(U,M.T)

In [181]:
pred = return_most_popular(predict,n_popular=20)

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [175]:
train_result = return_most_popular(V_train,n_popular = 20)

KeyboardInterrupt: 

In [176]:
sorted(range(m), key=lambda k: V_train[0][k] ,reverse=True)

[11195,
 23812,
 32134,
 2889,
 9733,
 11814,
 13995,
 14362,
 19962,
 22433,
 33351,
 36979,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 1

In [177]:
sorted(range(m), key=lambda k: V_test[0][k] ,reverse=True)

[37714,
 23084,
 6655,
 7279,
 13374,
 19901,
 21267,
 31379,
 39278,
 42069,
 45281,
 48270,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 1